# Variables

In [ ]:
TEMPERATURE=0.7
LANGUAGE = "spanish"

In [ ]:
INTRODUCE_STORY_PROMPT = "Generate a welcome message for the interactive story with a new setting, introducing the player's role, environment, key characters, and hinting at the main conflict. End with a question asking what the player will do. Do not present possible alternatives, let the player create his own."

In [ ]:
LANGUAGE_PROMPT = f"Answer always using the language {LANGUAGE}"

In [ ]:
EXAMPLE1 = "Example 1: You are a warrior in a medieval town, your sister recently died at the hands of an evil sorcerer. You are currently heading to the market to complete an errand for a friend, where you find a stranger sitting on a table and misteriously looking at you. What will you do?"
EXAMPLE2 = "Example 2: You are a pirate in Blackbeard's ship. The morning was going as usual, with the salty odor and calm waters, until you hear a stomp and see a giant tentacle going into the water. You ask another tripulant but he didn't hear or see anything. What will you do?"
EXAMPLE3 = "Example 3: You are a spaceship captain in the hunt for the infamous thief Lauren DeHugh, your tripulation follows you with pride and loyalty, but recently the moods have been weird, you suspect that the new passenger may have something to do, but it could also be nothing. Currently, you need to check the map and then you have a few minutes of spare time. What do you want to do?"

In [ ]:
DYNAMIC_PROMPT = INTRODUCE_STORY_PROMPT + EXAMPLE1 + EXAMPLE2 + EXAMPLE3 + LANGUAGE_PROMPT

In [ ]:
NARRATOR_SYSINT = (
    "system",
    "You are the narrator of an interactive story where the player's choices directly influence the progression and outcome of the narrative. "
    "Begin with an engaging introduction: set the stage with vivid, sensory details; describe the setting, introduce key characters, and hint at the main conflict. "
    "Speak directly to the player using 'you' to draw them into the story. As the story unfolds, organically introduce decision points where you ask the player what will he do. Leave the decision to the player, do not present choices"
    "Reflect the consequences of the player's choices, leading to multiple possible endings."
    "If the narrative reaches a point where the character dies, end with 'The End.' If the story concludes naturally, finish with 'The End.'"
)

In [ ]:
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' --retries 3

# Interactive Adventure

## Google API Key

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

In [ ]:
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

## System Prompt

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

In [ ]:
class StoryState(TypedDict):
    messages: Annotated[list, add_messages]
    finished: bool

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=TEMPERATURE)

## Define roles and actions

In [ ]:
from typing import Literal

In [ ]:
def narrator_node(state: StoryState) -> StoryState:
    if not state["messages"]:
        # Provide an extra prompt so that the LLM generates a dynamic welcome message.
        dynamic_prompt = ("human", DYNAMIC_PROMPT)
        output = llm.invoke([NARRATOR_SYSINT, dynamic_prompt])
    else:
        output = llm.invoke([NARRATOR_SYSINT] + state["messages"])
    print("\n********")
    print("Narrator:", output.content)
    return state | {"messages": [output]}

def player_node(state: StoryState) -> StoryState:
    print("\n********")
    user_input = input("You: ")
    if user_input in {"q", "quit", "exit", "goodbye"}:
        state["finished"] = True
    return state | {"messages": [("human", user_input)]}

def maybe_exit_player_node(state: StoryState) -> Literal["narrator", "__end__"]:
    if state.get("finished", False):
        return END
    else:
        return "narrator"


def maybe_exit_narrator_node(state: StoryState) -> Literal["player", "__end__"]:
    last_message = state["messages"][-1].content
    # If "The End." appears anywhere in the narrator's message, end the conversation.
    if "The End." in last_message:
        state["finished"] = True
        return END
    else:
        return "player"

In [ ]:
#from langchain_core.messages.ai import AIMessage
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

In [ ]:
graph_builder = StateGraph(StoryState)

# Add nodes
graph_builder.add_node("narrator", narrator_node)
graph_builder.add_node("player", player_node)

# Add edges
graph_builder.add_edge(START, "narrator")
graph_builder.add_conditional_edges("player", maybe_exit_player_node)
graph_builder.add_conditional_edges("narrator", maybe_exit_narrator_node)
story_graph = graph_builder.compile()

# Display Graph
Image(story_graph.get_graph().draw_mermaid_png())

In [ ]:
config = {"recursion_limit": 100}
#state = story_graph.invoke({"messages": []}, config)

# Gradio UI

In [ ]:
!pip install gradio -q

In [ ]:
import gradio as gr

In [ ]:
import gradio as gr
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# 1) Generamos SIN streaming el mensaje inicial para mostrarlo al cargar
first_ai = llm.invoke([
    SystemMessage(content=NARRATOR_SYSINT[1]),
    HumanMessage(content=DYNAMIC_PROMPT)
]).content
initial_history = [("", first_ai)]

# 2) Preparamos el LLM en modo streaming para el resto de la conversación
streaming_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", streaming=True, temperature=TEMPERATURE)

def stream_generate(user_input, history):
    # Si el usuario quiere salir, añadimos "The End." y desactivamos el textbox
    if user_input.lower() in {"q", "quit", "exit", "goodbye"}:
        history = history + [(user_input, "The End.")]
        yield history, history, gr.update(interactive=False, value="")
        return

    # Construimos la lista de mensajes para el LLM
    msgs = [SystemMessage(content=NARRATOR_SYSINT[1])]

    if history and history[0][0] == "":
        # Contexto inicial: reinyectamos sólo la salida AI
        msgs.append(AIMessage(content=history[0][1]))
    else:
        # Contexto normal: alternamos Human/AI por cada par
        for h, a in history:
            msgs.append(HumanMessage(content=h))
            msgs.append(AIMessage(content=a))

    # Añadimos el nuevo input del usuario
    msgs.append(HumanMessage(content=user_input))

    # Lo agregamos al historial con respuesta vacía por ahora
    history = history + [(user_input, "")]
    partial = ""
    # Stream token a token
    for token in streaming_llm.stream(msgs):
        partial += token.content
        history[-1] = (user_input, partial)
        # limpiamos el textbox en cada paso
        yield history, history, gr.update(interactive=True, value="")

    # Al terminar, si aparece "The End." desactivamos
    end_flag = "The End." in partial
    yield history, history, gr.update(interactive=not end_flag, value="")

def reset_chat():
    # Vuelve a generar un primer mensaje dinámico
    new_ai = llm.invoke([
        SystemMessage(content=NARRATOR_SYSINT[1]),
        HumanMessage(content=DYNAMIC_PROMPT)
    ]).content
    new_hist = [("", new_ai)]
    return new_hist, new_hist, gr.update(interactive=True, value="")

with gr.Blocks() as demo:
    chatbot   = gr.Chatbot(value=initial_history)
    state     = gr.State(initial_history)
    txtbox    = gr.Textbox(
        placeholder="Escribe tu acción...",
        container=False,
        autoscroll=True,
        scale=7
    )
    btn_reset = gr.Button("Start Again")

    # Al enviar el textbox, stream_generate devuelve 3 salidas:
    # (chatbot, state, txtbox)
    txtbox.submit(
        fn=stream_generate,
        inputs=[txtbox, state],
        outputs=[chatbot, state, txtbox]
    )

    # Al click en Start Again, reset_chat regenera el primer mensaje
    btn_reset.click(
        fn=reset_chat,
        inputs=[],
        outputs=[chatbot, state, txtbox],
        queue=False
    )

    demo.launch(share=True, debug=True)